## Imports

In [10]:
from ydata_profiling import ProfileReport

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots

In [11]:
df = pd.read_csv('data/data_clean.csv')
# df['cep_macro_region_code'] = df['cep_macro_region_code'].apply(lambda x: 'Macro '+str(x))
df.head()

,codigo_rota,status_tracking,rota_inicio,rota_final,hora_entrega,sq_plan,cep,distancia,distancia_rota,remessa,transportadora,veiculo,cep_macro_region_code,cep_sub_region_code,dia_entrega,time_span_entrega,total_entregas_rota,total_entregas_macro_regiao,max_dist_between_deliver
0,1,Delivered,2022-11-01 09:50:07,2022-11-02 08:47:56,2022-11-01 10:24:48,2,859,9.54,106.395,11894-84230,Transportadora A,Medio,8,5,1,10,23,3114,11.12
1,1,Delivered,2022-11-01 09:50:07,2022-11-02 08:47:56,2022-11-01 10:47:40,3,877,11.12,106.395,48346-11538,Transportadora A,Medio,8,7,1,10,23,3114,11.12
2,1,Delivered,2022-11-01 09:50:07,2022-11-02 08:47:56,2022-11-01 11:01:24,4,877,2.92,106.395,55669-65391,Transportadora A,Medio,8,7,1,11,23,3114,11.12
3,1,Delivered,2022-11-01 09:50:07,2022-11-02 08:47:56,2022-11-01 11:10:52,5,877,1.31,106.395,87726-35913,Transportadora A,Medio,8,7,1,11,23,3114,11.12
4,1,Delivered,2022-11-01 09:50:07,2022-11-02 08:47:56,2022-11-01 11:17:05,6,877,0.09,106.395,78072-89320,Transportadora A,Medio,8,7,1,11,23,3114,11.12


In [12]:
def region_to_uf(macro, sub):
    if (macro == 1):
        return 'SP'
    elif (macro == 2):
        if (sub <= 8):
            return 'RJ'
        elif (sub == 9):
            return 'ES'
    elif (macro == 3):
        return 'MG'
    elif (macro == 5):
        if (sub <= 6):
            return 'PE'
        elif (sub == 7):
            return 'AL'
        elif (sub == 8):
            return 'PB'
        elif (sub == 9):
            return 'RN'
    elif (macro == 7):
        if (sub <= 3):
            return 'DF'
        elif (sub == 3 or sub == 4):
            return 'GO'
    elif (macro == 8):
        if (sub <= 7):
            return 'PR'
        else:
            return 'SC'
    else:
        return 'MG'

FileNotFoundError: [Errno 2] No such file or directory: '../data/all_stocks_5yr.csv'

In [ ]:
# Creating columns to be used in the mapping graphs
df['Pais'] = 'Brasil'
df['UF'] = df.apply(lambda x: region_to_uf(x['cep_macro_region_code'], x['cep_sub_region_code']), axis=1)
df.head()
# df.to_csv('./data/data_clean_ceps.csv', index=False)

# Gráficos

## 1º Gráfico

Bubble Chart

Envolve a visualização de três atributos do dataset, sendo eles:
- número de entregas;
- macroregião do CEP;
- máxima distância entre entregas;
- distância total da rota;

In [ ]:
df = pd.read_csv('../data/data_clean.csv')

hover_data = {
    'Distância da rota (km)': df['distancia_rota'].astype(str)+' km',
    'Distância máxima entre entregas (km)': df['max_dist_between_deliver'].astype(str)+' km',
    'cep_macro_region_code': False,
}

labels={
    'distancia_rota': 'Distância da rota (km)',
    'max_dist_between_deliver': 'Distância máxima entre entregas (km)',
    'total_entregas_rota': 'Total de entregas na rota',
    'cep_macro_region_code': 'Macro região',
}

fig = px.scatter(df, x="distancia_rota", y="max_dist_between_deliver", size="total_entregas_rota",
        hover_name="codigo_rota", color="total_entregas_rota", facet_col="cep_macro_region_code",
        log_x=False, size_max=35, range_x=[10, 250], range_y=[-2, 34.8], hover_data=hover_data, labels=labels)

# change color attribute to total_entregas_rota ou fazer color por turno
# color=df["time_span_entrega"].astype(str)

fig.update_traces(
    marker=dict(
        opacity=0.7
    ))

fig.update_layout(
    hoverlabel=dict(
        font_size=16,
    )
)

fig.show()

## 2º Gráfico
Violin + scatter plot


Envolve a visualização de tais atributos
- x
- y
- z

In [ ]:
fig = px.violin(df, y="tip", x="smoker", color="sex", box=True, points="all",
          hover_data=df.columns)
fig.show()

## 3º Gráfico

Histograma de probabilidade acumulada da hora do dia por macro região

In [107]:
df = pd.read_csv('./data/data_clean.csv')

# Agrupamento por macro região
df_by_mr_ts = [0,0,0,0,0,0,0,0,0]
df_hist_acc = df[['time_span_entrega', 'cep_macro_region_code']]
df_ts_percent = df_hist_acc.groupby(['time_span_entrega', 'cep_macro_region_code']).size().reset_index(name='count')

for macro_region_code in df['cep_macro_region_code'].unique():
    df_by_mr_ts[macro_region_code] = pd.DataFrame()

    ts_percent_by_mr = df_ts_percent[df_ts_percent['cep_macro_region_code'] == macro_region_code]

    df_by_mr_ts[macro_region_code]['time_span_entrega'] = ts_percent_by_mr['time_span_entrega']
    df_by_mr_ts[macro_region_code]['accumulated'] = ts_percent_by_mr['count'].cumsum()
    df_by_mr_ts[macro_region_code]['acc_percent'] = df_by_mr_ts[macro_region_code]['accumulated'] / df_by_mr_ts[macro_region_code]['accumulated'].max()

labels={
    'x': 'Hora do dia',
    'acc_percent': 'Percentual acumulado de entregas',
}

hov_data = {
    'acc_percent': ':.2f',
}

# Selecionar macro região a ser visualizada
macro_region_to_be_visualized = 2
fig = px.bar(df_by_mr_ts[macro_region_to_be_visualized], x=df_by_mr_ts[macro_region_to_be_visualized]["time_span_entrega"].astype(str),
            y="acc_percent", text=df_by_mr_ts[macro_region_to_be_visualized]['acc_percent'].astype(float).round(2),
            labels=labels, color='acc_percent', hover_data=hov_data)

fig.show()


### Histograma de probabilidade acumulada do horário da entrega entre todas regiões

In [ ]:
df = pd.read_csv('./data/data_clean.csv')

df_hist_acc = df[['time_span_entrega']]
df_grouped_ts = df_hist_acc.groupby(['time_span_entrega']).size().reset_index(name='count')
df_grouped_ts['accumulated'] = df_grouped_ts['count'].cumsum()
df_grouped_ts['acc_percent'] = df_grouped_ts['accumulated'] / df_grouped_ts['accumulated'].max()
df_grouped_ts

labels={
    'x': 'Hora do dia',
    'acc_percent': 'Percentual acumulado de entregas',
}

hov_data = {
    'acc_percent': ':.2f',
}

fig = px.bar(df_grouped_ts, x=df_grouped_ts["time_span_entrega"].astype(str),
            y="acc_percent", text=df_grouped_ts['acc_percent'].astype(float).round(2),
            labels=labels, color='acc_percent', hover_data=hov_data)

fig.show()

# Arthur

In [ ]:
rota_columns = ['codigo_rota',
                'rota_inicio',
                'rota_final',
                'distancia_rota',
                'transportadora',
                'veiculo',
                'total_entregas_rota',
                'max_dist_between_deliver']

rotas = df[rota_columns].drop_duplicates()
rotas['rota_inicio'] = pd.to_datetime(rotas['rota_inicio'])
rotas['rota_final'] = pd.to_datetime(rotas['rota_final'])
rotas['tempo_rota'] = rotas['rota_final'] - rotas['rota_inicio']
rotas['horas_rota'] = rotas['tempo_rota'].dt.total_seconds() / 3600
rotas

,codigo_rota,rota_inicio,rota_final,distancia_rota,transportadora,veiculo,total_entregas_rota,max_dist_between_deliver,tempo_rota,horas_rota
0,1,2022-11-01 09:50:07,2022-11-02 08:47:56,106.395,Transportadora A,Medio,23,11.12,0 days 22:57:49,22.963611
23,2,2022-11-02 00:08:13,2022-11-02 00:10:06,35.176,Transportadora B,Pequeno,1,20.83,0 days 00:01:53,0.031389
24,3,2022-11-02 13:08:07,2022-11-03 13:16:15,97.101,Transportadora A,Medio,37,15.53,1 days 00:08:08,24.135556
61,4,2022-11-02 07:24:29,2022-11-03 06:56:30,82.437,Transportadora A,Medio,37,20.97,0 days 23:32:01,23.533611
98,5,2022-11-02 15:39:15,2022-11-02 22:13:46,68.686,Transportadora A,Medio,44,16.70,0 days 06:34:31,6.575278
...,...,...,...,...,...,...,...,...,...,...
9716,311,2022-11-30 11:04:30,2022-12-01 11:09:03,76.903,Transportadora A,Medio,29,5.82,1 days 00:04:33,24.075833
9745,312,2022-11-30 09:28:12,2022-11-30 15:15:11,85.156,Transportadora A,Medio,40,20.13,0 days 05:46:59,5.783056
9785,313,2022-11-30 11:53:58,2022-11-30 18:03:59,71.250,Transportadora A,Medio,39,16.53,0 days 06:10:01,6.166944
9824,314,2022-11-30 14:04:29,2022-12-01 13:07:38,89.661,Transportadora A,Medio,40,25.51,0 days 23:03:09,23.052500


In [19]:
filtered_rotas = rotas[rotas['horas_rota'] <= 100]

fig = px.scatter(df, x='codigo_rota', y='distancia', title='Distância percorrida vs. Sequência de entrega planejada')
fig.show()

fig = px.scatter(df, x='sq_plan', y='distancia', title='Distância percorrida vs. Sequência de entrega planejada')
fig.show()

# Grafico de correlação entre tempo total da rota e distancia percorrida
fig = px.scatter(filtered_rotas, x='horas_rota', y='distancia_rota', title='Distância percorrida vs. Tempo total da rota')
fig.show()


# Grafico de correlação entre tempo total da rota e numero de entregas
fig = px.scatter(filtered_rotas, x='horas_rota', y='total_entregas_rota', title='Número de entregas vs. Tempo total da rota')
fig.show()